In [ ]:
pip uninstall openai

Found existing installation: openai 0.28.0
Uninstalling openai-0.28.0:
  Would remove:
    /usr/local/bin/openai
    /usr/local/lib/python3.10/dist-packages/openai-0.28.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/openai/*
Proceed (Y/n)? Y
  Successfully uninstalled openai-0.28.0


In [1]:
pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00


In [95]:
import json
import openai
import os
import time
from itertools import product

In [96]:
#Set OpenAI API key

openai.api_key = 'your-api-key'

In [97]:
# Define profile attributes
age_groups = ["18-22 years old", "23-26 years old"]
genders = ["Man", "Woman"]
ethnicities = ["Asian", "White"]
educations = ["Bachelor’s degree", "Master’s degree", "Professional degree"]

In [98]:
# Generate all possible profiles including domain expertise
profiles = list(product(age_groups, genders, ethnicities, educations))

In [99]:
print(len(profiles))

24


In [100]:
# Define the questionnaire
questionnaire = [
    {
        "question": "What is your preferred development environment?",
        "answers": {
            "option_1": "Windows",
            "option_2": "macOS",
            "option_3": "Linux",
            "option_4": "Other:"
        }
    },
    {
        "question": "How do you learn to code? Please select all that apply.",
        "answers": {
            "option_1": "Online Courses or Certification",
            "option_2": "Books",
            "option_3": "School (i.e., University, College, etc.)",
            "option_4": "Coding Bootcamp",
            "option_5": "Other:"
        }
    },
    {
        "question": "What is the biggest challenge you face as a developer?",
        "answers": {
            "option_1": "Keeping up with new technologies",
            "option_2": "Work-life balance",
            "option_3": "Understanding existing codebases",
            "option_4": "Time management"
        }
    },
    {
        "question": "When choosing a programming language for a new project you prioritize:",
        "answers": {
            "option_1": "The language's performance and scalability",
            "option_2": "The development team's familiarity with the language",
            "option_3": "The language's community support and ecosystem",
            "option_4": "The specific requirements of the project"
        }
    },
    {
        "question": "How do you communicate effectively with teammates to collaborate while adhering to the timelines?",
        "answers": {
            "option_1": "Use project management tools to assign tasks and track progress ensuring everyone is aware of deadlines.",
            "option_2": "Schedule regular meetings for updates and coordination but keep them concise to avoid taking too much time away from work.",
            "option_3": "Rely on informal chats and emails for quick updates trusting team members to manage their time efficiently.",
            "option_4": "Implement a combination of written documentation for clarity and regular check-ins for personal engagement and immediate feedback."
        }
    },
    {
        "question": "How do you ensure that you stay up-to-date with industry changes as a software developer? Please select all that apply.",
        "answers": {
            "option_1": "Regularly read industry blogs, websites, and journals to learn about the latest trends and technologies.",
            "option_2": "Attend workshops, conferences, and webinars to gain insights from industry leaders and peers.",
            "option_3": "Participate in online courses and certification programs to acquire new skills and knowledge.",
            "option_4": "Engage in open-source projects and coding communities to get hands-on experience with cutting-edge technologies."
        }
    },
    {
        "question": "How do you balance between innovation and meeting project deadlines?",
        "answers": {
            "option_1": "Prioritize innovation over deadlines",
            "option_2": "Focus solely on meeting deadlines",
            "option_3": "Allocate specific time for research and innovation",
            "option_4": "Adjust project scopes to include innovation"
        }
    },
    {
        "question": "Software development contributes to societal challenges by:. Please select all that apply",
        "answers": {
            "option_1": "Improving healthcare through technology, such as telemedicine platforms and health tracking apps",
            "option_2": "Enhancing education with e-learning platforms, interactive learning tools, and virtual classrooms",
            "option_3": "Addressing climate change with environmental monitoring systems, carbon footprint calculators, and energy efficiency applications",
            "option_4": "Supporting economic development through fintech solutions, e-commerce platforms, and digital marketplaces"
        }
    },
    {
        "question": "A company uses an AI system to monitor its employees' productivity. The AI suggests firing an employee based on low productivity, but the employee is going through a tough personal time. How should the company proceed?",
        "answers": {
            "option_1": "Follow the AI’s suggestion and fire the employee",
            "option_2": "Ignore the AI’s suggestion and keep the employee",
            "option_3": "Use the AI’s suggestion as one of several factors in the decision",
        }
    },
    {
        "question": "You're assigned to a project that requires expertise in a programming language you're not familiar with. The deadline is tight. What is your strategy to tackle this situation?",
        "answers": {
            "option_1": "Spend time upfront to learn the basics of the language",
            "option_2": "Use your existing knowledge and adapt as you go",
            "option_3": "Seek assistance from a colleague who knows the language",
            "option_4": "Suggest reallocating the project to someone with the necessary expertise",
        }
    },
    {
        "question": "You discover a critical bug right before a software release. What immediate action do you take?",
        "answers": {
            "option_1": "Proceed with the release as planned and fix the bug in a subsequent patch",
            "option_2": "Delay the release to assess and fix the bug, ensuring the product meets quality standards",
            "option_3": "Release a limited beta version to a select group of users for further testing while addressing the bug",
            "option_4": "Communicate with stakeholders about the issue and seek their input on how to proceed",
        }
    },
    {
        "question": " In a professional setting, if your team is supposed to deliver a SaaS product within a two-day timeframe, and you discover a critical bug in the software, how would you address and handle this situation?",
        "answers": {
            "option_1": "Ignore the bug and proceed with the delivery as planned, with the intention of fixing the bug in the next update.",
            "option_2": "Inform your team leader or project manager about the bug immediately, and work collaboratively to assess its impact and decide on the next steps",
            "option_3": "Delay the delivery without informing stakeholders to buy time for fixing the bug",
            "option_4": "Document the bug and send an email to the client explaining the issue, suggesting a new timeline for delivery",
            "option_5": "Prioritize the bug based on its severity and impact, and if it's critical, reallocate resources to fix it immediately, even if this means pushing back the delivery date",
        }
    },

]


In [101]:
def call_gpt(prompt, temperature=0.0):
    try:
        start_time = time.time()  # Time before the API call

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": prompt}],
            temperature=temperature,
        )

        end_time = time.time()  # Time after the API call
        duration = end_time - start_time  # Duration of the API call

        print(f"API call duration: {duration} seconds")  # Print or log the duration

        return response.choices[0].message['content'], duration
    except Exception as e:
        print(f"Error in generating response: {e}")
        return None, 0

In [103]:
print(len(profiles))

24


In [107]:
# Generate responses for each profile
all_responses = []
prompt_id = 1

for profile in profiles:
    # ... [Profile setup] ...
    age, gender, ethnicity, education = profile
    profile_desc = f" of age: {age}, gender: {gender}, {ethnicity}, with a {education}"

    # Construct a combined prompt for all questions for the profile
    combined_prompt = f"Imagine you are {profile_desc}. Answer the following questions and strictly choose from the given options only.\n\n"
    for i, q in enumerate(questionnaire, start=1):
        combined_prompt += f"Q{i}: {q['question']}\n"
        for key, option in q['answers'].items():
            combined_prompt += f"   {key}: {option}\n"

    # Call the GPT model once for the combined prompt
    combined_response, duration = call_gpt(combined_prompt)

    # Process the combined response to extract answers for each question
    responses_for_profile = []
    for i, q in enumerate(questionnaire, start=1):
        answer_start = f"Q{i}:"
        answer_end = f"Q{i+1}:" if i < len(questionnaire) else None
        start_idx = combined_response.find(answer_start) + len(answer_start)
        end_idx = combined_response.find(answer_end) if answer_end else len(combined_response)
        answer = combined_response[start_idx:end_idx].strip()

        # Determine the option_ids and handle 'Other' option
        option_ids = [key for key, value in q["answers"].items() if value in answer]
        other_response = answer.split('Other:')[1].strip() if 'Other:' in answer else None

        responses_for_profile.append({
            "Question": q['question'],
            "Answer": answer,
            "option_ids": option_ids,
            "other_response": other_response
        })

    all_responses.append({
        "prompt_id": prompt_id,
        "prompt": combined_prompt,
        "profile": profile_desc,
        "Responses": responses_for_profile
    })
    prompt_id += 1


API call duration: 5.390301942825317 seconds
API call duration: 5.304432153701782 seconds
API call duration: 5.405527114868164 seconds
API call duration: 5.295734643936157 seconds
API call duration: 7.087076187133789 seconds
API call duration: 4.6090662479400635 seconds
API call duration: 5.232895374298096 seconds
API call duration: 6.046212196350098 seconds
API call duration: 5.438427686691284 seconds
API call duration: 6.056016206741333 seconds
API call duration: 5.501573801040649 seconds
API call duration: 4.867854118347168 seconds
API call duration: 5.450608491897583 seconds
API call duration: 4.792653322219849 seconds
API call duration: 5.197298765182495 seconds
API call duration: 5.41353702545166 seconds
API call duration: 4.771698474884033 seconds
API call duration: 4.149969100952148 seconds
API call duration: 8.291820526123047 seconds
API call duration: 5.028151750564575 seconds
API call duration: 4.992710590362549 seconds
API call duration: 4.883029937744141 seconds
API call d

In [108]:
# Save the responses in the specified format
def save_json(data, filename) :
  with open(filename, 'w', encoding='utf-8') as f :
    json.dump(data, f, indent=4)

In [109]:
save_json(all_responses, 'ecs_260_all_profiles_responses.json')